Working with a dataset that contains movie reviews labeled as positive or negative.

In [4]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


df = pd.read_csv("imdb-dataset/IMDB Dataset.csv")

print(df.head())

#Affichage des colonnes du dataset

print(df.columns)
















                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
Index(['review', 'sentiment'], dtype='object')


In [ ]:
#Effectuez des opérations de nettoyage des textes (suppression de la ponctuation, des stop words, mise en minuscules, etc.)

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
df['cleaned_reviews'] = df['review'].apply(lambda x: ' '.join([word for word in word_tokenize(x.lower()) if word not in stop_words and word not in string.punctuation]))

print(df['cleaned_reviews'].head())


In [5]:
#encodage des labels pour transformer les labels catégoriels en valeurs numériques

le = LabelEncoder()
df['sentiment_encoded'] = le.fit_transform(df['sentiment'])



In [6]:

#Divisez les données en ensembles d'entraînement et de test.
#Using train_test_split to divide  data into training and test sets.

#feature_column is  review column      and    label_column is  sentiment  


# Features and labels
X = df['review']
y = df['sentiment']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the sizes of the splits
print(f'Training set size: {X_train.shape[0]}')
print(f'Test set size: {X_test.shape[0]}')


Training set size: 40000
Test set size: 10000


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Utilisation de pad_sequences de Keras pour que toutes les séquences aient la même longueur.

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=100)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)

#vérification de la taille des séquences

print("Taille de X_train_pad:", X_train_pad.shape)
print("Taille de X_test_pad:", X_test_pad.shape)

# entrainement de 3 Mdèles 



Taille de X_train_pad: (40000, 100)
Taille de X_test_pad: (10000, 100)
